In [1]:
!pip install transformers
!pip install peft
!pip install accelerate
!pip install torchvision
!pip install datasets
!pip install pillow
!pip install evaluate
!pip install evaluate bert_score nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import os
import pandas as pd
import torch
from transformers import ViltProcessor, ViltConfig, ViltForQuestionAnswering
from peft import PeftModel
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score
from evaluate import load
from tqdm import tqdm



In [3]:
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"



In [4]:
# Load saved model, processor, and config
root = "/content/drive/Othercomputers/MyMacBookPro/VR_PROJECT_2"
# Load base ViLT model and processor directly (no adapters)
model_name = "dandelin/vilt-b32-finetuned-vqa"
processor = ViltProcessor.from_pretrained(model_name)
model = ViltForQuestionAnswering.from_pretrained(model_name)

# Move model to device
model.to(device)
model.eval()



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/136k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/470M [00:00<?, ?B/s]

ViltForQuestionAnswering(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=76

In [6]:
# Load test data
test_csv = os.path.join(root, "vqa_test_cleaned.csv")
df = pd.read_csv(test_csv)

# Initialize metric libraries
bertscore = load("bertscore")
meteor = load("meteor")

# Lists to store predictions and ground truth
true_answers = []
predicted_answers = []



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [7]:
# Evaluation loop
for _, row in tqdm(df.iterrows(), total=len(df)):
    try:
        image_path = os.path.join(root, "images", row["path"])
        image = Image.open(image_path).convert("RGB")
        image = image.resize((384, 384))
        question = row["generated_question"]
        true_answer = row["generated_answer"].lower()

        # Preprocess
        inputs = processor(images=image, text=question, return_tensors="pt").to(device)

        # Inference
        with torch.no_grad():
            outputs = model(**inputs)
            pred_id = outputs.logits.argmax(-1).item()
            predicted_answer = model.config.id2label[pred_id].lower()

        true_answers.append(true_answer)
        predicted_answers.append(predicted_answer)
    except Exception as e:
        print(f"Skipping row due to error: {e}")
        continue



  5%|▌         | 169/3318 [02:28<39:25,  1.33it/s]

Skipping row due to error: The size of tensor a (100) must match the size of tensor b (40) at non-singleton dimension 1


  7%|▋         | 221/3318 [03:11<38:26,  1.34it/s]

Skipping row due to error: The size of tensor a (41) must match the size of tensor b (40) at non-singleton dimension 1


 23%|██▎       | 771/3318 [10:52<28:19,  1.50it/s]

...

 99%|█████████▉| 3301/3318 [46:08<00:12,  1.39it/s]

Skipping row due to error: The size of tensor a (52) must match the size of tensor b (40) at non-singleton dimension 1


100%|█████████▉| 3312/3318 [46:16<00:03,  1.51it/s]

Skipping row due to error: The size of tensor a (101) must match the size of tensor b (40) at non-singleton dimension 1


100%|██████████| 3318/3318 [46:22<00:00,  1.19it/s]


In [8]:
# Compute standard metrics
accuracy = accuracy_score(true_answers, predicted_answers)
f1 = f1_score(true_answers, predicted_answers, average='weighted')



In [9]:
# Compute advanced metrics
bertscore_results = bertscore.compute(predictions=predicted_answers, references=true_answers, model_type="bert-base-uncased")
# bartscore_results = bartscore.compute(predictions=predicted_answers, references=true_answers, model_type="facebook/bart-large")
meteor_results = meteor.compute(predictions=predicted_answers, references=true_answers)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [10]:
# Output results
print(f"\n--- Evaluation Results ---")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"BERTScore F1: {sum(bertscore_results['f1']) / len(bertscore_results['f1']):.4f}")
# print(f"BARTScore F1: {sum(bartscore_results['f1']) / len(bartscore_results['f1']):.4f}")
print(f"METEOR Score: {meteor_results['meteor']:.4f}")



--- Evaluation Results ---
Accuracy: 0.5458
F1 Score: 0.5190
BERTScore F1: 0.8668
METEOR Score: 0.2807
